In [ ]:
!pip install pendulum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import pendulum as pn

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [ ]:
# Скачивание и задание начальных данных
!gdown 19oOzbzkLT30p-I3_PEN3pnTmYl19pgtq
transactions = pd.read_csv('/content/transactions.csv', nrows = 1000000, sep = ',')

!gdown 1_X0rk6Tn61AXRhb6vPS3I3HC2PC9IDfp
genderTrain = pd.read_csv('/content/gender_train.csv', sep = ',')

!gdown 1oucWBfzlrQKD10PDohT_5JrxL-quH9A_
trMccCodes = pd.read_csv('/content/tr_mcc_codes.csv', sep = ';')

!gdown 1vRyq3WKezr094eY9Ie24XDo9yjS-7ZPj
trTypes = pd.read_csv('/content/tr_types.csv', sep = ';')

Downloading...
From: https://drive.google.com/uc?id=19oOzbzkLT30p-I3_PEN3pnTmYl19pgtq
To: /content/transactions.csv
100% 314M/314M [00:01<00:00, 247MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_X0rk6Tn61AXRhb6vPS3I3HC2PC9IDfp
To: /content/gender_train.csv
100% 99.9k/99.9k [00:00<00:00, 54.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oucWBfzlrQKD10PDohT_5JrxL-quH9A_
To: /content/tr_mcc_codes.csv
100% 14.9k/14.9k [00:00<00:00, 19.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vRyq3WKezr094eY9Ie24XDo9yjS-7ZPj
To: /content/tr_types.csv
100% 14.2k/14.2k [00:00<00:00, 19.5MB/s]


In [ ]:
numberOfLines = 1000

# Выбор произвольных строк
selectedTransactions = transactions.sample(numberOfLines)

# Задание правила выбора типов и непосредственно отбор
logicalResults = (trTypes['tr_description'].str.contains('POS')) | (trTypes['tr_description'].str.contains('ATM'))
suitableTypes = trTypes['tr_type'][logicalResults]

# Поиск количества транзакций, тип которых подходит и подсчёт числа таких транзакций
suitableTransactions = selectedTransactions.tr_type.isin(suitableTypes)
numberOfSuitableTransactions = len(suitableTransactions[suitableTransactions == True])

# Подсчёт доли транзакций
print(numberOfSuitableTransactions / numberOfLines)

0.372


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
# Подсчёт частоты встречаемости всех типов транзакций
allFrequences = transactions['tr_type'].value_counts()

# Отбор первых 10
topTenFrequences = allFrequences.head(10)

# Вывод нужных транзакций
trTypes[trTypes['tr_type'].isin(topTenFrequences.index)]

,tr_type,tr_description
39,7010,Взнос наличных через АТМ (в своем тер.банке)
47,7030,Перевод на карту (с карты) через АТМ (в предел...
59,7070,Перевод на карту (с карты) через Мобильный бан...
60,7071,Перевод на карту (с карты) через Мобильный бан...
98,1010,Покупка. POS ТУ СБ РФ
99,1030,Оплата услуги. Банкоматы СБ РФ
100,1100,Покупка. ТУ Россия
101,1110,Покупка. POS ТУ Россия
106,2010,Выдача наличных в АТМ Сбербанк России
129,2370,Списание с карты на карту по операции <перевод...


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
# Выделение нужных столбцов
allClientsWithAmount = transactions[['customer_id', 'amount']]

# Разделение приходов и расходов
clientsWithIncome = allClientsWithAmount[allClientsWithAmount['amount'] > 0]
clientsWithWithdrawals = allClientsWithAmount[allClientsWithAmount['amount'] < 0]

# Получение сумм для приходов и расходов
clientsWithTotalIncome = clientsWithIncome.groupby('customer_id').sum()
clientsWithTotalWithdawal = clientsWithWithdrawals.groupby('customer_id').sum()

# Поиск клиента с максимальной суммой приходов по карте и клиента с максимальной суммой расходов по карте
clientWithBiggestTotalIncome = clientsWithTotalIncome.idxmax().item()
clientWithBiggestTotalWithdrawal = clientsWithTotalWithdawal.idxmin().item()

# Вывод найденных выше клиентов
print('Client with biggest total income:', clientWithBiggestTotalIncome)
print('Client with biggest total withdrawal:', clientWithBiggestTotalWithdrawal)

# Поиск и вывод для двух ранее найденных клиентов модуля разницы между суммой расходов и суммой приходов 
print('Difference for client with biggest total income:', abs(clientsWithTotalWithdawal['amount'][clientWithBiggestTotalIncome] - clientsWithTotalIncome['amount'][clientWithBiggestTotalIncome]))
print('Difference for client with biggest total withdrawal:', abs(clientsWithTotalWithdawal['amount'][clientWithBiggestTotalWithdrawal] - clientsWithTotalIncome['amount'][clientWithBiggestTotalWithdrawal]))

Client with biggest total income: 70780820
Client with biggest total withdrawal: 70780820
Difference for client with biggest total income: 2498067091.6
Difference for client with biggest total withdrawal: 2498067091.6


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
# Отбор десяти транзакций
topTenTransactions = transactions[transactions['tr_type'].isin(list(topTenFrequences.index))]

# Поиск среднего арифметического и медианы для найденных транзакций 
meanForTopTenTransactions = topTenTransactions.groupby('tr_type').amount.mean()
medianForTopTenTransactions = topTenTransactions.groupby('tr_type').amount.median()

# Вывод полученных результатов
print('Mean and median for top 10 transactions:')
pd.concat([meanForTopTenTransactions, medianForTopTenTransactions], axis = 1, keys = ['Mean', 'Median'])

Mean and median for top 10 transactions:


,Mean,Median
tr_type,,
1010,-19784.748640,-7411.52
1030,-5320.980222,-2245.92
1100,-44061.827262,-10188.26
1110,-32119.330371,-11207.57
2010,-136077.629325,-44918.32
2370,-205418.249032,-44918.32
7010,276391.789596,112295.79
7030,86104.332909,13951.52
7070,65569.831700,11319.42


In [ ]:
# Поиск транзакций клиентов
secondTaskFirstClientTransactions = transactions[transactions['customer_id'] == clientWithBiggestTotalIncome]
secondTaskSecondClientTransactions = transactions[transactions['customer_id'] == clientWithBiggestTotalWithdrawal]

# Поиск среднего арифметического и медианы для транзакций клиентов
meanForFirstClient = secondTaskFirstClientTransactions.groupby('tr_type').amount.mean()
medianForFirstClient = secondTaskFirstClientTransactions.groupby('tr_type').amount.median()

meanForSecondClient = secondTaskSecondClientTransactions.groupby('tr_type').amount.mean()
medianForSecondClient = secondTaskSecondClientTransactions.groupby('tr_type').amount.median()

In [ ]:
# Вывод полученных результатов
print('Mean and median for the first client:')
pd.concat([meanForFirstClient, medianForFirstClient], axis = 1, keys = ['Mean', 'Median'])

Mean and median for the first client:


,Mean,Median
tr_type,,
2010,-2.941056e+06,-3368873.660
2330,-2.382398e+06,-2245915.770
2370,-2.218505e+06,-2245915.770
6110,1.756293e+04,10028.010
7020,1.465129e+04,10319.985
7021,2.850067e+04,28500.670
7030,1.406196e+04,8848.910
7031,1.921036e+04,12992.620
7034,1.430648e+04,14306.480


In [ ]:
# Вывод полученных результатов
print('Mean and median for the second client:')
pd.concat([meanForSecondClient, medianForSecondClient], axis = 1, keys = ['Mean', 'Median'])

Mean and median for the second client:


,Mean,Median
tr_type,,
2010,-2.941056e+06,-3368873.660
2330,-2.382398e+06,-2245915.770
2370,-2.218505e+06,-2245915.770
6110,1.756293e+04,10028.010
7020,1.465129e+04,10319.985
7021,2.850067e+04,28500.670
7030,1.406196e+04,8848.910
7031,1.921036e+04,12992.620
7034,1.430648e+04,14306.480


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, genderTrain, how='left')
transactions = pd.merge(transactions, trMccCodes, how='inner')
transactions = pd.merge(transactions, trTypes, how='inner')
transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
meansOfGenderWithdrawals = transactions[transactions['amount'] < 0].groupby('gender').amount.mean()

print(round(abs(meansOfGenderWithdrawals.array[0] - meansOfGenderWithdrawals.array[1]), 2))

32718.05


In [ ]:
meansOfGenderWithdrawals = transactions[transactions['amount'] > 0].groupby('gender').amount.mean()

print(round(abs(meansOfGenderWithdrawals.array[0] - meansOfGenderWithdrawals.array[1]), 2))

63366.57


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:
max_income = transactions[transactions['amount'] > 0].groupby(['gender', 'tr_type']).amount.sum()

minimumIncomesZeroGender = max_income[0].sort_values().head(10)
minimumIncomesFirstGender = max_income[1].sort_values().head(10)

print(minimumIncomesZeroGender)
print(minimumIncomesFirstGender)

tr_type
4051     1122.96
4210     2245.92
4110     2245.92
2370     2335.75
7075     6737.75
2110    22459.16
1010    31532.66
4100    40334.88
2210    68315.82
1210    83644.87
Name: amount, dtype: float64
tr_type
2020      2245.92
4110      4491.84
1010     35317.03
8100     64682.37
6000     79280.83
7041     87590.72
2370    113194.15
7015    121279.45
2110    179673.26
2010    224591.59
Name: amount, dtype: float64


In [ ]:
pd.merge(minimumIncomesZeroGender, minimumIncomesFirstGender, on = ['tr_type'])

,amount_x,amount_y
tr_type,,
4110,2245.92,4491.84
2370,2335.75,113194.15
2110,22459.16,179673.26
1010,31532.66,35317.03


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [ ]:
maxWithdrawals = transactions[transactions['amount'] < 0].groupby(['gender', 'mcc_code']).amount.sum()

cathegoriesWithBiggestDifference = abs(maxWithdrawals[0] - maxWithdrawals[1]).sort_values(ascending = False)

topTenCathegories = cathegoriesWithBiggestDifference.head(10)

trMccCodes[trMccCodes['mcc_code'].isin(topTenCathegories.index)]


,mcc_code,mcc_description
22,4829,Денежные переводы
62,5511,"Легковой и грузовой транспорт — продажа, серви..."
64,5533,Автозапчасти и аксессуары
65,5541,Станции техобслуживания
75,5661,Обувные магазины
90,5812,"Места общественного питания, рестораны"
116,5977,Магазины косметики
123,6010,Финансовые институты — снятие наличности вручную
124,6011,Финансовые институты — снятие наличности автом...
162,7995,Транзакции по азартным играм


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

# Задание на защиту


Найдите 10 самых частых категорий, по которым траты совершаются в ночное время. Выведите их названия и суммы трат по убыванию этих сумм.

In [ ]:
transactions['hours'] = transactions.tr_datetime.str.extract('( \d\d)') 

neededTransactions = transactions[['mcc_code', 'hours', 'amount']]

groupsOfTransactions = neededTransactions[neededTransactions['hours'].str.contains("00|01|02|03|04|05|06")].groupby('mcc_code').size().sort_values(ascending = False).reset_index(name="counter").head(10)

sumsOfTransactions =  neededTransactions[neededTransactions['hours'].str.contains("00|01|02|03|04|05|06")].groupby('mcc_code').amount.sum().reset_index(name="summary")

result = pd.merge(groupsOfTransactions, sumsOfTransactions)

result = pd.merge(result, trMccCodes)

result = result.sort_values(by=['summary'])

result[['mcc_description', 'summary']]

,mcc_description,summary
3,Финансовые институты — снятие наличности автом...,-1.484930e+09
5,Денежные переводы,-1.169168e+09
1,"Бакалейные магазины, супермаркеты",-6.945035e+08
4,Станции техобслуживания,-2.555692e+08
8,"Места общественного питания, рестораны",-1.490091e+08
2,"Звонки с использованием телефонов, считывающих...",-8.260529e+07
9,Аптеки,-7.429261e+07
7,"Рестораны, закусочные",-5.980176e+07
6,"Различные продовольственные магазины — рынки, ...",-5.111227e+07
0,Финансовые институты — снятие наличности вручную,1.439876e+09
